In [ ]:
import numpy as np
import pandas as pd
import antropy as ent
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.fft import fft, fftfreq 
import seaborn as sns
from scipy import signal
import seaborn as sns
from scipy import stats
from os.path import exists

sns.set_context('paper')

# Definiendo las funciones requeridas

In [ ]:

# Gráfica de la evolución de la entropía para diferentes subjects
def plotEntropyAnalisis(subjects, phaseslist, ent_kind, metrics, step, overlap):
    bandnames = ['delta', 'theta', 'alpha', 'beta', 'gamma', 'all bands']
    print("loading the data")
    # we load the data
    for i,subject in enumerate(subjects):
        phases = phaseslist[i]
        comparison = phases[0]+phases[1]
        for metric in metrics:
            fileroot = 'results/entropy_'+subject+'_'+ent_kind+'_'+str(metric)+'_'+str(step)+'_'+str(overlap)
            ent = pd.read_excel(fileroot+'_ent.xlsx')
            ent['metric'] = metric
            ent['subject'] = subject
            stat = pd.read_excel(fileroot+'_'+phases[0]+'_stats.xlsx')
            mask = pd.notna(stat['diff'])
            stat = stat[mask]
            stat['metric'] = metric
            stat['subject'] = subject
            stat_prepost = pd.read_excel(fileroot+'_'+comparison+'_stats.xlsx')
            mask = pd.notna(stat_prepost['diff'])
            stat_prepost = stat_prepost[mask]
            stat_prepost['metric'] = metric
            stat_prepost['subject'] = subject
            try:
                ent_data = pd.concat([ent_data, ent])
                stat_data = pd.concat([stat_data, stat])
                stat_prepost_data = pd.concat([stat_prepost_data, stat_prepost])
            except:
                ent_data = ent
                stat_data = stat
                stat_prepost_data = stat_prepost
            # creating a new column with the subject + session information
            ent_data['info'] = ent_data['subject']+'_'+ent_data['sesion'].astype('str')+'_'+ent_data['fase']
            stat_data['info'] = stat_data['subject']+'_'+stat_data['sesion'].astype('str')
            stat_prepost_data['info'] = stat_prepost_data['subject']+'_'+stat_prepost_data['sesion'].astype('str')

    print("loaded the data")      
    
    # A figure will be created for each band and each EEG. In each case a figure showing the evolution of the calculated
    # entropy as a function of the used metric. The session number, or control number, will be used as the hue
    
    
    for band in bandnames:
        for eeg in ['EEG1', 'EEG2']:
            ###################################################################################
            ################## GENERATING THE PLOT
            if subjects[-1]!='C':
                fig = plt.figure(figsize=(10,16))
                spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                spec.update(left=0.05, right= 0.99, top = 0.95, bottom = 0.52, wspace=0.1, hspace = 0.25)
                spec2 = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                spec2.update(left=0.05, right= 0.99, top = 0.46, bottom = 0.03, wspace=0.1, hspace = 0.25)
            else:
                fig = plt.figure(figsize=(10,8))
                spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                spec.update(left=0.05, right= 0.99, top = 0.9, bottom = 0.1, wspace=0.1, hspace = 0.25)

                

            line1 = []
            line2 = []
            line3 = []
            line4 = []
            line1.append(plt.subplot(spec[0,0]))
            line2.append(plt.subplot(spec[1,0], sharey = line1[0]))
            if subjects[-1]!='C':
                line3.append(plt.subplot(spec2[0,0]))
                line4.append(plt.subplot(spec2[1,0], sharey = line3[0]))


            ax = [line1,line2,line3,line4]
  
            #ax.set_title('Evolución entropía\n modo: '+key+', señal: '+eeg)
            if ent_kind == 'sampen':
                fig.suptitle('Sample Entropy; band: '+band+'; eeg: '+eeg)
            elif ent_kind == 'permen':
                fig.suptitle('Permutation Entropy; band: '+band+'; eeg: '+eeg)

            for i, fase in enumerate(['pre', 'post']):
                mask = ((ent_data['fase']==fase) & (ent_data['eeg']==eeg) & (ent_data['banda']==band)) | ((ent_data['eeg']==eeg) & (ent_data['banda']==band) & (ent_data['subject']=='C'))
                print(len(ent_data[mask]))
                sns.boxplot(data = ent_data[mask], x='metric', y='entropia', hue ='info', ax = ax[i][0])
                ax[i][0].legend(fontsize=7, loc = 'upper right')
                ax[i][0].set_title("Evolución Entropía "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band + '; fase: '+ fase)

            subnames = '_'.join(subjects)
            if subjects[-1]!='C':                
                mask = (stat_data['eeg']==eeg) & (stat_data['bands']==band) & (stat_data['subject']!='C')
                sns.barplot(data = stat_data[mask], x='metric', y='diff', hue = 'sesion', ax = ax[2][0])
                ax[2][0].axhline(y=0)
                ax[2][0].legend(fontsize=7, loc = 'upper left')
                ax[2][0].set_title("cambio "+ subject[-1] +" PRE-PRE "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)
                #sns.boxplot(data = statsPREPOST[mask], x='bands', y='diff', hue = 'sesion', ax = ax[j][3])
                #ax[j][3].legend(fontsize=4)
                mask = (stat_prepost_data['eeg']==eeg) & (stat_prepost_data['bands']==band) & (stat_prepost_data['subject']!='C')
                sns.barplot(data = stat_prepost_data[mask], x='metric', y='diff', hue = 'sesion', ax = ax[3][0])
                ax[3][0].axhline(y=0)
                ax[3][0].legend(fontsize=7, loc = 'upper left')
                ax[3][0].set_title("cambio "+ subject[-1] +" PRE-POST "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)

            filename = 'images/entropy_'+subnames+'_'+ent_kind+'_'+eeg+'_'+band+'_'+str(overlap)+'.png'
            plt.savefig(filename, dpi = 400)
            plt.close()
            
# Gráfica de la evolución de la entropía para diferentes subjects
def plotEntropyAnalisisMS(subjects, phaseslist, metrics, scales, step, overlap):
    ent_kind = 'msen'
    bandnames = ['delta', 'theta', 'alpha', 'beta', 'gamma', 'all bands']
    # we load the data
    for i,subject in enumerate(subjects):
        phases = phaseslist[i]
        comparison = phases[0]+phases[1]
        for metric in metrics:
            for scale in scales:
                print(metric,scale)
                fileroot = 'results/entropy_'+subject+'_'+ent_kind+'_'+str(metric)+'_'+str(scale)+'_'+str(step)+'_'+str(overlap)
                ent = pd.read_excel(fileroot+'_ent.xlsx')
                ent['metric'] = metric
                ent['subject'] = subject
                ent['scale'] = scale
                print(len(ent))
                stat = pd.read_excel(fileroot+'_'+phases[0]+'_stats.xlsx')
                stat['metric'] = metric
                stat['subject'] = subject
                stat['scale'] = scale
                print(len(stat))
                stat_prepost = pd.read_excel(fileroot+'_'+comparison+'_stats.xlsx')
                stat_prepost['metric'] = metric
                stat_prepost['subject'] = subject
                stat_prepost['scale'] = scale
                print(len(stat_prepost))
                try:
                    ent_data = pd.concat([ent_data, ent])
                    stat_data = pd.concat([stat_data, stat])
                    stat_prepost_data = pd.concat([stat_prepost_data, stat_prepost])
                except:
                    ent_data = ent
                    stat_data = stat
                    stat_prepost_data = stat_prepost
                # creating a new column with the subject + session information
                ent_data['info'] = ent_data['subject']+'_'+ent_data['sesion'].astype('str')+'_'+ent_data['fase']
                stat_data['info'] = stat_data['subject']+'_'+stat_data['sesion'].astype('str')
                stat_prepost_data['info'] = stat_prepost_data['subject']+'_'+stat_prepost_data['sesion'].astype('str')

                    
    print("loaded the data")
    # A figure will be created for each band, each EEG, and each metric. In each case a figure showing the evolution of the calculated
    # entropy as a function of the used metric. The session number, or control number, will be used as the hue
    
    for metric in metrics:
        for band in bandnames:
            for eeg in ['EEG1', 'EEG2']:
                ###################################################################################
                ################## GENERATING THE PLOT
                if subjects[-1]!='C':
                    fig = plt.figure(figsize=(10,16))
                    spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                    spec.update(left=0.05, right= 0.99, top = 0.95, bottom = 0.52, wspace=0.1, hspace = 0.25)
                    spec2 = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                    spec2.update(left=0.05, right= 0.99, top = 0.46, bottom = 0.03, wspace=0.1, hspace = 0.25)
                else:
                    fig = plt.figure(figsize=(10,8))
                    spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                    spec.update(left=0.05, right= 0.99, top = 0.9, bottom = 0.1, wspace=0.1, hspace = 0.25)



                line1 = []
                line2 = []
                line3 = []
                line4 = []
                line1.append(plt.subplot(spec[0,0]))
                line2.append(plt.subplot(spec[1,0], sharey = line1[0]))
                if subjects[-1]!='C':
                    line3.append(plt.subplot(spec2[0,0]))
                    line4.append(plt.subplot(spec2[1,0], sharey = line3[0]))


                ax = [line1,line2,line3,line4]

                #ax.set_title('Evolución entropía\n modo: '+key+', señal: '+eeg)
                if ent_kind == 'sampen':
                    fig.suptitle('Sample Entropy; band: '+band+'; eeg: '+eeg+'; metric: '+ metric)
                elif ent_kind == 'permen':
                    fig.suptitle('Permutation Entropy; band: '+band+'; eeg: '+eeg+'; metric: '+ metric)

                for i, fase in enumerate(['pre', 'post']):
                    mask = ((ent_data['fase']==fase) & (ent_data['eeg']==eeg) & (ent_data['banda']==band) & (ent_data['metric']==metric)) | ((ent_data['metric']==metric) & (ent_data['eeg']==eeg) & (ent_data['banda']==band) & (ent_data['subject']=='C'))
                    sns.boxplot(data = ent_data[mask], x='scale', y='entropia', hue ='info', ax = ax[i][0])
                    ax[i][0].legend(fontsize=7, loc = 'upper right')
                    ax[i][0].set_title("Evolución Entropía "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band + '; fase: '+ fase)

                subnames = '_'.join(subjects)
                if subjects[-1]!='C':                
                    mask = (stat_data['eeg']==eeg) & (stat_data['bands']==band) & (stat_data['metric']==metric) & (stat_data['subject']!='C')
                    sns.barplot(data = stat_data[mask], x='scale', y='diff', hue = 'sesion', ax = ax[2][0])
                    ax[2][0].axhline(y=0)
                    ax[2][0].legend(fontsize=7, loc = 'upper left')
                    ax[2][0].set_title("cambio "+ subject[-1] +" PRE-PRE "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)
                    #sns.boxplot(data = statsPREPOST[mask], x='bands', y='diff', hue = 'sesion', ax = ax[j][3])
                    #ax[j][3].legend(fontsize=4)
                    mask = (stat_prepost_data['eeg']==eeg) & (stat_prepost_data['bands']==band) & (stat_prepost_data['subject']!='C') & (stat_prepost_data['metric']==metric)
                    sns.barplot(data = stat_prepost_data[mask], x='scale', y='diff', hue = 'sesion', ax = ax[3][0])
                    ax[3][0].axhline(y=0)
                    ax[3][0].legend(fontsize=7, loc = 'upper left')
                    ax[3][0].set_title("cambio "+ subject[-1] +" PRE-POST "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)

                filename = 'images/entropy_'+subnames+'_'+ent_kind+'_'+eeg+'_'+band+'_'+str(metric)+'_'+str(overlap)+'.png'
                plt.savefig(filename, dpi = 400)
                plt.close()

# Plotting

## Control evolution

## Paciente 1 Evolución

In [ ]:
plotEntropyAnalisis(['C', 'P1'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['C', 'P1'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['C', 'P1'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

## Paciente 2 Evolución

In [ ]:
plotEntropyAnalisis(['C', 'P2'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['C', 'P2'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['C', 'P2'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

## Paciente 3 Evolución

In [ ]:
plotEntropyAnalisis(['C', 'P3'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['C', 'P3'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['C', 'P3'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

### Paciente 4 Evolución

In [ ]:
plotEntropyAnalisis(['C', 'P4'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['C', 'P4'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['C', 'P4'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

### Paciente 5 Evolución

In [ ]:
plotEntropyAnalisis(['C', 'P5'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['C', 'P5'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['C', 'P5'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

### Paciente 6 Evolución

In [ ]:
plotEntropyAnalisis(['C', 'P6'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['C', 'P6'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['C', 'P6'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

# Paciente 7

In [ ]:
plotEntropyAnalisis(['C', 'P7'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['C', 'P7'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['C', 'P7'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)